# Ajuste de hiperparámetros: Optimización Bayesiana

En muchos algoritmos de inteligencia artificial, existen dos tipos de parámetros: aquellos que son inicializados aleatoriamente y pueden ser actualizados durante el entrenamiento y aquellos que no pueden ser estimados durante el entrenamiento sino que deben ser establecidos al inicio del proceso de aprendizaje, pues son parte de la configuración del modelo.
Los parámetros no estimables se conocen como hiperparámetros.

Los hiperparámetros son importantes pues pueden afectar el desempeño del modelo. Por ejemplo, en el caso de los árboles de decisión, el número de niveles de profundidad del árbol es un hiperparámetro. Si el árbol es muy profundo, puede sobreajustarse a los datos de entrenamiento, mientras que si es muy plano, puede no ser capaz de aprender patrones complejos. Por lo tanto, es importante encontrar el número óptimo de niveles de profundidad para el árbol.

En este notebook, veremos cómo podemos ajustar los hiperparámetros de un modelo de aprendizaje automático utilizando la optimización bayesiana.

La optimización bayesiana es un método de optimización que se basa en la teoría de la probabilidad. En lugar de probar todas las combinaciones de hiperparámetros, la optimización bayesiana utiliza un modelo probabilístico para estimar la probabilidad de qué combinación de hiperparámetros es la mejor. El modelo probabilístico se actualiza a medida que se prueban nuevas combinaciones de hiperparámetros, lo que permite que el algoritmo se centre en las combinaciones de hiperparámetros que tienen más probabilidades de ser las mejores.

### Ajuste de hiperparámetros: teoría simplificada
El objetivo de la optimización de hipreparámetros es encontrar la combinación de hiperparámetros $\mathbb{A}$ que maximiza alguna de las métricas explicadas con anterioridad o minimiza el error de clasificación. Puede representarse matemáticamente como:
$$ \mathbb{A}= arg \hspace{0.8mm} min _{x \in \pi}\hspace{0.8mm} f(x)$$

donde $\pi$ es el espacio de búsqueda de hiperparámetros y $f(x)$ es la función objetivo que se desea optimizar.

En cualquier problema $\pi$ debe ser definido a priori.

### Optimización bayesiana: teoría simplificada
En la optimización Bayesiana, se mantiene un registro de las evaluaciones de la función objetivo $f(x)$ para cada combinación de hiperparámetros $x \in \pi$. El registro se utiliza para construir un modelo probabilístico que se actualiza a medida que se evalúan nuevas combinaciones de hiperparámetros. El modelo probabilístico se utiliza para estimar la probabilidad de que una combinación de hiperparámetros sea la mejor. La función probabilistica para elegir las combinaciones de hiperparámetros es:
$$ P(Puntuación | x)$$

### Inconvenientes de la optimización de hiperparámetros

* La optimización de hiperparámetros es un proceso costoso en términos de tiempo y recursos computacionales, ya que deben probarse una serie de combinaciones de hiperparámetros. 

* La mejora en el desempeño del modelo puede ser marginal y es algo del cual nunca se puede estar seguro.

* La convergencia de la optimización bayesiana puede ser lenta, especialmente si el modelo probabilístico no es capaz de capturar la relación entre las combinaciones de hiperparámetros y la función objetivo.

* Deben conocerse los hiperparámetros del modelo a optimizar, ya que el hecho de añadir o eliminar hiperparámetros puede afectar el proceso de optimización aumentando el tiempo de convergencia.

En este notebook veremos cómo podemos ajustar los hiperparámetros del modelo de xenofobia utilizando la optimización bayesiana.

### Librerias útiles

##### Hugging Face
Se trata de una comunidad y plataforma de ciencia de datos que ofrece una gran cantidad de herramientas para la creación y evaluación de modelos de aprendizaje profundo. Entre ellas destacan [[Omer Mahmood @ Towards Data Science](https://towardsdatascience.com/whats-hugging-face-122f4e7eb11a#:~:text=Hugging%20Face%20is%20a%20community,(OS)%20code%20and%20technologies.)]:
* Herramientas que permiten a los usuarios construir, entrenar y desplegar modelos basados en código abierto
* Un lugar donde una amplia comunidad de científicos de datos, ingenieros de aprendizaje profundo e investigadores pueden reunirse para compartir ideas, obtener apoyo, contribuir a los proyectos e incluso compartir sus modelos entrenados o puros.

<figure>
    <img src="./assets/images/hug.png"
         alt="Hugging Face logo">
    <figcaption>Hugging Face logo</figcaption>
</figure>

No se encontró ningún modelo entrenado para la tarea de interés, por lo que se entrenaron y probaron 12 modelos a modo de seleccionar el mejor de ellos. La selección de estos 12 modelos consistió en tomar aquellos que estuviesen entrenados para una tarea similar a la xenofobia (en este caso fue el discurso de odio) y/o que hayan sido entrenados para comprender el idioma español.

De este estudio el modelo seleccionado fue [RoBERTuito-base-uncased](https://arxiv.org/abs/2111.09453)

##### PyTorch
Es una librería de aprendizaje automático de código abierto *que acelera el camino desde la creación de prototipos de investigación hasta el despliegue de producción [[PyTorch](https://pytorch.org/)].*

<figure>
    <img src="./assets/images/pytorch.png"
         alt="PyTorch logo">
    <figcaption>PyTorch logo</figcaption>
</figure>

Algunos de los modelos disponibles en Hugging Face se encuentran implementados sobre esta librería. De este modo, algunas de las herramientas disponibles en PyTorch son fácilmente adaptables con Hugging Face, esto nos permitirá añadir o modificar la estructura de una red neuronal, así como su comportamiento, permitiendo al usuario implementar varias funciones personalizadas.

##### Scikit learn
Se trata de una librería de código abierto enfocada en proveer herramientas de aprendizaje de máquinas tales como modelos estadísticos y matemáticos, así como métricas de evaluación comunes en algoritmos de aprendizaje de máquinas.
<figure>
    <img src="./assets/images/scikit.png"
         alt="scikit-learn logo"
         style="max-width: 20%; height: auto">
    <figcaption>scikit-learn logo</figcaption>
</figure>

Esta librería nos permitirá implementar de manera sencilla las métricas de evaluación del modelo de interés.

##### Adaptive Experimentation Platform (AX)
AX es una plataforma para optimizar cualquier tipo de experimento, incluyendo experimentos de aprendizaje automático, pruebas A/B y simulaciones [[AX](https://ax.dev/docs/why-ax.html)].
<figure>
    <img src="./assets/images/ax.svg"
         alt="AX logo"
         style="max-width: 20%; height: auto">
    <figcaption>AX logo</figcaption>
</figure>
AX nos permitirá optimizar los hiperparámetros del modelo a través del métodoBayesiano, así como evaluar el desempeño del modelo en términos de métricas de evaluación (en conjunto con Scikit-Learn).

In [1]:
#Imports

#HuggingFace library
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import Dataset, Value, ClassLabel, Features

#PyTorch Neural Networks
import torch
import torch.nn as nn

#data reading
import pandas as pd

#math
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
import os, random, re

#scikit-learn metrics
from sklearn.metrics import (
    confusion_matrix, recall_score, accuracy_score, recall_score, precision_score, f1_score, classification_report
)

#Bayesian optimization
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.plot.contour import plot_contour_plotly
import itertools

### Carga y configuración del tokenizador

In [2]:
#set model name
model_name = "pysentimiento/robertuito-base-uncased"
#Load tokenizer
#Un tokenizer es un objeto que convierte una secuencia de caracteres en una secuencia de números.
#es una especie de filtro que prepara el texto para que el modelo lo pueda entender.
tokenizer = AutoTokenizer.from_pretrained(model_name)
#Add special tokens to the tokenizer
tokenizer.add_tokens(['@usuario', 'url', 'hashtag', 'emoji'])
tokenizer.model_max_length = 128

### Lectura de datos y construcción del conjunto de datos

In [3]:
data_train = pd.read_csv('./assets/data/train.csv')
data_valid = pd.read_csv('./assets/data/valid.csv')

In [5]:
def tokenize(batch):
        """Tokenize text in current mini batch. This is a util function for get_dataset_from_dataframes function

        Args:
            batch (batched datasets.arrow_dataset.Dataset)
        
        Returns:
            [datasets.arrow_dataset.Dataset]: Mapped text-label dataset
        """
        return tokenizer(batch['text'], padding=False, truncation=True)

def format_dataset(dataset):
    """Map text-label for specific dataset from pandas. This is a util function for get_dataset_from_dataframes function

    Args:
        dataset (datasets.arrow_dataset.Dataset): Dataset from pandas DataFrame

    Returns:
        [datasets.arrow_dataset.Dataset]: Mapped text-label dataset
    """
    def get_labels(examples):
        return {'labels': examples['label']}

    dataset = dataset.map(get_labels)
    return dataset

#Features to map insto dataset
features = Features({
    'text': Value('string'),
    'label': ClassLabel(num_classes=2, names=['ok', 'hateful'])
    })

train_dataset = Dataset.from_pandas(data_train, features=features)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=8)
train_dataset = format_dataset(train_dataset)

valid_dataset = Dataset.from_pandas(data_valid, features=features)
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=8)
valid_dataset = format_dataset(valid_dataset)

#to be able to use batched training, we need to use a data collator
data_collator = DataCollatorWithPadding(tokenizer, padding='longest')


  0%|          | 0/875 [00:00<?, ?ba/s]

0ex [00:00, ?ex/s]

  0%|          | 0/125 [00:00<?, ?ba/s]

0ex [00:00, ?ex/s]

### Construcción de métricas
Dado que solo nos concentraremos en la métrica de valor-F1 para la clase más desbalanceada, se implementará una función que nos permita calcular únicamente el valor-F1. En caso de interesarse por alguna otra métrica, debe implementarse la función correspondiente, tal y como se hizo en el notebook de entrenamiento del modelo de xenofobia.

In [6]:
def compute_metrics(p):
    """Compute Accuracy, Precision, Recall and F1 metrics

    Args:
        p ([List]): List with calculated logits by model and real label per sample

    Returns:
        [dict]: dict with calculated metrics
    """
    pred, labels = p
    #Get class with most probability
    pred = np.argmax(pred, axis=-1)
    f1_scr = f1_score(y_true=labels, y_pred=pred, pos_label=1, average='binary')

    return {'f1_cls1': f1_scr}

### Construcción de la función de entrenamiento y del modelo
Puesto que para cada conjunto de parámetros se entrenará desde cero el modelo, se implementará una función que permita entrenar el modelo con los parámetros de interés. Esta función se encargará de entrenar el modelo y evaluarlo con las métricas de interés.

In [7]:
def init_model():
    ''' init model and config it using global variable model_name'''
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, return_dict=True, num_labels=2)
    
    model.config.id2label = {
            0: 'ok',
            1: 'hateful',
        }
    id2label = {
                0: 'ok',
                1: 'hateful',
            }
    label2id = {v:k for k,v in id2label.items()}
    model.config.label2id = label2id
    model.resize_token_embeddings(len(tokenizer))
    
    return model

Algunos de los hiperparámetros que se pueden ajustar son:
* Número de épocas: número de veces que se recorre el conjunto de datos de entrenamiento.
* Tasa de aprendizaje: tasa de actualización de los pesos de la red neuronal.
* Los pesos de "importancia" de cada clase: se puede ajustar la importancia de cada clase para que el modelo se enfoque en predecir mejor aquellas clases que se encuentren más desbalanceadas.
* El warmup ratio: se puede ajustar la tasa de aprendizaje de manera que se empiece con una tasa de aprendizaje baja y se vaya incrementando hasta llegar a la tasa de aprendizaje deseada.
* El weight decay: se puede ajustar la tasa de regularización de los pesos de la red neuronal. (un método para evitar el sobreajuste)

Para poder modificar los pesos de "importancia" de cada clase, es necesario modificar una instancia de la clase Trainer, la cual se encarga de entrenar el modelo. Para ello, debe usarse la función *compute_loss* que se encuentra en la clase Trainer, la cual se encarga de calcular la pérdida del modelo. Se añade a la función de pérdida entropía cruzada una lista de pesos de "importancia" para cada clase.


In [8]:
def train_model(model, train_data, parameters):
    ''' Train model'''
    def get_optimizer_weights(parameters):
        ''' use parameters dict to get tensor of weights for each class '''
        weights_class0 = parameters.get('opt_weights_cls0',0.7543)
        weights_class1 = parameters.get('opt_weights_cls1',1.0)
        return torch.tensor([weights_class0, weights_class1]) 
    
    class MyTrainer(Trainer):
        ''' redefine class Trainer such that it's possible to use weights in loss function '''
        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get('labels')
            outputs = model(**inputs)
            logits = outputs.get('logits')
            loss_fct = nn.CrossEntropyLoss(weight = get_optimizer_weights(parameters).cuda())
            loss = loss_fct(logits, labels)
            return (loss, outputs) if return_outputs else loss    
    
    """
    Training arguments. 
    learning_rate, weight_decay, warmup_ratio, epochs, opt_weights_cls0, opt_weights_cls1,
    adam_beta2 and adam_epsilon are parameters to optimize.
    """ 
    training_args = TrainingArguments(
        output_dir='./robertuito/',
        num_train_epochs=parameters.get('epochs', 3),
        seed=3,
        warmup_ratio=parameters.get('warm_up_ratio', 0.0),
        evaluation_strategy='no',
        save_strategy='no',
        do_eval=False,
        logging_dir='./logs',
        load_best_model_at_end=False,
        learning_rate = parameters.get('lr', 5e-5),
        weight_decay = parameters.get('weight_decay', 0.0),
        group_by_length=True,
        )
    
    trainer_args = {
        'model': model,
        'args': training_args,
        'train_dataset': train_dataset,
        'eval_dataset': valid_dataset,
        'data_collator': data_collator,
        'tokenizer': tokenizer,
    }
    #instanciate new Trainer class and Train without evaluation per epoch
    trainer = MyTrainer(**trainer_args, compute_metrics=compute_metrics)
    trainer.train()
    
    return trainer

def evaluate_model(model, valid_data):
    ''' Evaluate model using Trainer class from huggingface and validation data '''
    return model.evaluate(valid_data)

def train_evaluate(parameterization):
    ''' Train and evaluate a bayesian optimization trial '''
    trained_model = train_model(model=init_model(), train_data=train_dataset, parameters=parameterization)
    metric = evaluate_model(trained_model, valid_dataset)
    del trained_model
    #change metric name if not interested in recall
    return metric['f1_cls1']

### Optimización bayesiana
Para realizar la optimización se usa el método optimize de la libreria AX. Este método recibe como parámetros la función a optimizar, así como los parámetros a optimizar y sus rangos de valores. El método optimize regresa una lista de diccionarios, cada uno de los cuales contiene los valores de los parámetros que se obtuvieron al optimizar la función de interés.

In [ ]:
#Run bayesian optimization with defined parameters and its characteristics
#see https://ax.dev/api/core.html#parameter for more info about parameters
best_parameters, values, experiment, model = optimize(
    parameters=[
        {'name': 'lr', 'type': 'range', 'bounds': [1e-6, 8e-5], 'log_scale': True, 'value_type':'float'},
        {'name': 'weight_decay', 'type': 'range', 'bounds': [0.005, 0.50], 'log_scale': True, 'value_type':'float'},
        {'name': 'warm_up_ratio', 'type': 'range', 'bounds': [0.0, 0.8], 'log_scale': False, 'value_type':'float'},
        {'name': 'epochs', 'type': 'range', 'bounds': [1, 6], 'value_type':'int'},
        {'name': 'opt_weights_cls0', 'type': 'range', 'bounds': [0.60, 1.0], 'log_scale': True, 'value_type':'float'},
        {'name': 'opt_weights_cls1', 'type': 'range', 'bounds': [1.0, 3.5], 'log_scale': True, 'value_type':'float'},        
    ],
    evaluation_function=train_evaluate,
    #change metric name if not interested in recall
    objective_name='f1_cls1',
)

print(best_parameters)

Al terminar la optimización bayesiana, se imprime el mejor conjunto de parámetros encontrado, como en la siguiente imagen:
<figure>
    <img src="./assets/images/bayesian_opt.png"
         alt="best parameters found"
         style="max-width: 60%; height: auto">
</figure>


Para guardar los parámetros encontrados para cada iteración así como sus resultados, se usa el siguiente código

In [ ]:
#get and save data from every trial
data = experiment.fetch_data()
params_by_arm = pd.DataFrame([experiment.arms_by_name[i].parameters for i in experiment.arms_by_name]).reset_index()
params_by_arm.rename(columns={'index':'trial_index'}, inplace=True)
data = data.df.merge(params_by_arm, on='trial_index')
data.to_csv('./assets/data/parameters_per_run_recall.csv', index=False)

Se obtendrá un resultado similar al siguiente
<figure>
    <img src="./assets/images/opt_results.png"
         alt="parameters and results bayesian optimization"
         style="max-width: 60%; height: auto">
</figure>